In [4]:
print(__doc__)

from time import time
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter

from sklearn import manifold, datasets

# Next line to silence pyflakes. This import is needed.
Axes3D
# FILENAME here

filename="../preprocessed/amlodipine.csv"

f = open(filename)
f.readline()  # skip the header
data =  np.loadtxt(fname = f, delimiter = ',',dtype='double')

label = data[:,0]
X = data[:, 1:data.shape[1]-1]
# Dataset has Label at index 1

Automatically created module for IPython interactive environment


#### Normalizing to Zero Mean Unit Variance

In [5]:
mean = X.mean(axis=0)
std = X.std(axis=0)
X = (X - mean) / std

### Validate No. of neighbors & Components

#### Restriction: n_neighbors > [n_components * (n_components + 3) / 2]

** The no. of neighbors is to be validated **

In [6]:
n_neighbors = 6
n_components = 2

## Locally Linear Embeddings and Variations

In [7]:

fig = plt.figure(figsize=(18, 12))
plt.suptitle("Trying out different Manifold Techniques", fontsize=14)

## Some compatibility code below per Sklearn Example
try:
    # compatibility matplotlib < 1.0
    ax = fig.add_subplot(251, projection='3d')
    ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=label, cmap=plt.cm.Spectral)
    ax.view_init(4, -72)
except:
    ax = fig.add_subplot(251, projection='3d')
    plt.scatter(X[:, 0], X[:, 2], c=label, cmap=plt.cm.Spectral)

methods = ['standard', 'ltsa', 'hessian', 'modified']
labels = ['LLE', 'LTSA', 'Hessian LLE', 'Modified LLE']

for i, method in enumerate(methods):
    t0 = time()
    Y = manifold.LocallyLinearEmbedding(n_neighbors, n_components,
                                        eigen_solver='dense',
                                        method=method).fit_transform(X)
    t1 = time()
    print("%s: %.2g sec" % (methods[i], t1 - t0))

    ax = fig.add_subplot(252 + i)
    plt.scatter(Y[:, 0], Y[:, 1], c=label, cmap=plt.cm.Spectral)
    plt.title("%s (%.2g sec)" % (labels[i], t1 - t0))
    ax.xaxis.set_major_formatter(NullFormatter())
    ax.yaxis.set_major_formatter(NullFormatter())
    plt.axis('tight')


standard: 0.027 sec
ltsa: 0.044 sec
hessian: 0.11 sec
modified: 0.05 sec


## Isomap Learning

In [8]:
t0 = time()
Y = manifold.Isomap(n_neighbors, n_components).fit_transform(X)
t1 = time()
print("Isomap: %.2g sec" % (t1 - t0))
ax = fig.add_subplot(257)
plt.scatter(Y[:, 0], Y[:, 1], c=label, cmap=plt.cm.Spectral)
plt.title("Isomap (%.2g sec)" % (t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')

Isomap: 0.036 sec


(-12.368011806920746,
 20.088832106843213,
 -10.554476156998549,
 14.846899559524843)

## Multi-dimensional Scaling (MDS)

In [9]:

t0 = time()
mds = manifold.MDS(n_components, max_iter=100, n_init=1)
Y = mds.fit_transform(X)
t1 = time()
print("MDS: %.2g sec" % (t1 - t0))
ax = fig.add_subplot(258)
plt.scatter(Y[:, 0], Y[:, 1], c=label, cmap=plt.cm.Spectral)
plt.title("MDS (%.2g sec)" % (t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')

MDS: 0.13 sec


(-11.917518463391128,
 10.130642992383486,
 -10.420231856851236,
 10.937269647733363)

## Spectral Embeddings

In [10]:

t0 = time()
se = manifold.SpectralEmbedding(n_components=n_components,
                                n_neighbors=n_neighbors)
Y = se.fit_transform(X)
t1 = time()
print("SpectralEmbedding: %.2g sec" % (t1 - t0))
ax = fig.add_subplot(259)
plt.scatter(Y[:, 0], Y[:, 1], c=label, cmap=plt.cm.Spectral)
plt.title("SpectralEmbedding (%.2g sec)" % (t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')


SpectralEmbedding: 0.025 sec


(-0.34346141541171055,
 0.62957455389437444,
 -0.52102996715027017,
 0.59264008872546459)

## t-Distributed Stochastic Neighbor Embedding (t-SNE)

In [11]:
t0 = time()
tsne = manifold.TSNE(n_components=n_components, init='pca', random_state=0)
Y = tsne.fit_transform(X)
t1 = time()
print("t-SNE: %.2g sec" % (t1 - t0))
ax = fig.add_subplot(2, 5, 10)
plt.scatter(Y[:, 0], Y[:, 1], c=label, cmap=plt.cm.Spectral)
plt.title("t-SNE (%.2g sec)" % (t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')

t-SNE: 1.3 sec


(-304.31430089631027,
 207.99996705455413,
 -326.98457561205629,
 453.9475591139859)

In [12]:
plt.show()